<a href="https://www.kaggle.com/code/bunniebytes/ctd-assignment-7?scriptVersionId=291013058" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

try: 
    from thefuzz import process 
except ImportError: 
    !pip install thefuzz 
    from thefuzz import process

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/code-the-dream-assignment-6/name_and_address_3.csv
/kaggle/input/code-the-dream-assignment-6/name_and_address_0.csv
/kaggle/input/code-the-dream-assignment-6/name_and_address_2.csv
/kaggle/input/code-the-dream-assignment-6/name_and_address_1.csv
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 27.6 MB/s eta 0:00:00


## Task 1: How Would You Solve This?

1. Combine all the items into 1 data set
   * We know that there are 400 people and 200 addresses, some with same name but different address or same address but different person.
2. Create a new data set to add cleaned data into it
3. Sort data/group by name AND address using fuzz
   * Assign an id to the groups of people who have the same address
4. Determine correct spelling of name (majority spelling) and same with the address and zip.  Disregard the spaces without data
5. Move through to the phone numbers and find majority.  Disregard the spaces without data
6. Confirm size of the final data set and that there are no empty/invalid info

## Task 2: Load Dataset

In [2]:
names1 = pd.read_csv("/kaggle/input/code-the-dream-assignment-6/name_and_address_0.csv")
names2 = pd.read_csv("/kaggle/input/code-the-dream-assignment-6/name_and_address_1.csv")
names3 = pd.read_csv("/kaggle/input/code-the-dream-assignment-6/name_and_address_2.csv")
names4 = pd.read_csv("/kaggle/input/code-the-dream-assignment-6/name_and_address_3.csv")

display(names1.head())

df = pd.concat([names1, names2, names3, names4], ignore_index = True)
display(df.info())

,Name,Address,Zip,Phone
0,Jerry Bartlett,"009 Tristan Meadow Martinezbury, NV",97263.0,(616)477-6740x83474
1,Jennifer Gonzalez,"0098 Finley Valleys Port Michelleshire, ID",97726.0,6094074870
2,Jack Reed,"0098 Finley Valleys Port Michelleshire, ID",97726.0,432.845.9420
3,Mrs. Gail Perez,"03284 Carson Fields Acostaview, AK",NaN,+1-826-533-8606x22246
4,Andrew Martin,"03284 Carson Fields Acostaview, AK",45268.0,NaN


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1600 entries, 0 to 1599
Data columns (total 4 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   Name     1600 non-null   object 
 1   Address  1600 non-null   object 
 2   Zip      1261 non-null   float64
 3   Phone    1369 non-null   object 
dtypes: float64(1), object(3)
memory usage: 50.1+ KB


None

In [3]:
display(df.loc[df["Name"] == "Charles Smith"])
display(df.loc[(df["Address"] == "140 Larry Center Suite 787 West Charlesbury, MD")])

,Name,Address,Zip,Phone
12,Charles Smith,"05678 Mccullough Branch Elizabethshire, VA",6636.0,001-882-807-9438
37,Charles Smith,"1144 Martin Villages Apt. 102 Hebertport, NM",22162.0,(302)317-7064x056
93,Charles Smith,"25724 Mary Flat Apt. 920 Jamestown, GA",13131.0,001-486-462-7641
216,Charles Smith,"612 Walter Grove Suite 736 Morrismouth, RI",74843.0,412.401.8223
323,Charles Smith,"908 Kelsey Tunnel Suite 775 CasXXsandrabury, ID",42138.0,NaN
377,Charles Smith,USNV Johnson FPO AP,44100.0,+1-547-291-3943x705
412,Charles Smith,"05678 Mccullough Branch Elizabethshire, VA",6636.0,001-882-807-9438
437,Charles Smith,"1144 Martin Villages Apt. 102 Hebertport, NM",22162.0,(302)317-7064x056
493,Charles Smith,"25724 Mary Flat Apt. 920 Jamestown, GA",13131.0,001-486-462-7641
616,Charles Smith,"612 Walter Grove Suite 736 Morrismouth, RI",NaN,3856773171


,Name,Address,Zip,Phone
67,Jared Martinez,"140 Larry Center Suite 787 West Charlesbury, MD",44181.0,+1-658-356-4910x175
68,Margaret Khan,"140 Larry Center Suite 787 West Charlesbury, MD",44181.0,779.681.3822x3803
69,Jason Gill,"140 Larry Center Suite 787 West Charlesbury, MD",44181.0,+1-839-825-0522x68039
467,Jared Martinez,"140 Larry Center Suite 787 West Charlesbury, MD",NaN,(642)821-6095
469,Jason Gill,"140 Larry Center Suite 787 West Charlesbury, MD",44181.0,+1-839-825-0522x68039
867,Jared Martinez,"140 Larry Center Suite 787 West Charlesbury, MD",44181.0,400-734-3477x4170
868,Margaret Khan,"140 Larry Center Suite 787 West Charlesbury, MD",NaN,+1-678-470-4485x4317
869,Jason Gill,"140 Larry Center Suite 787 West Charlesbury, MD",44181.0,+1-839-825-0522x68039
1268,Margaret KhaXXn,"140 Larry Center Suite 787 West Charlesbury, MD",44181.0,NaN


## Task 3: Clean Up Spelling Errors

In [4]:
df_names = df.value_counts("Name")
names = list(df_names[df_names > 2].index)

display(df_names.head(10))

df["Name"] = df["Name"].map(lambda x : x if x in names else process.extractOne(x, names)[0])

df_addresses = df.value_counts("Address")
addresses = list(df_addresses[df_addresses > 2].index)

df["Address"] = df["Address"].map(lambda x : x if x in addresses else process.extractOne(x, addresses)[0])

Name
Charles Smith     23
Rebecca Carey     16
Pamela Costa      15
Jennifer Levy     15
Brandi Sanders    14
Thomas Jones      14
April Lee         12
Jacob Hardy       12
Samuel Brown      12
Alan Carpenter    12
Name: count, dtype: int64

## Task 4: Clean Up the Zip and Phone Columns

In [5]:
def fix_anomaly(group): 
    group_na = group.dropna() 
    if group_na.empty: 
        return group.values 
    mode = group_na.mode() 
    if mode.empty: 
        return group.values 
    return mode.iloc[0]

df['Zip'] = df.groupby(['Name', 'Address'], as_index=False)['Zip'].transform(fix_anomaly).reset_index(drop=True)
df['Phone'] = df.groupby(['Name', 'Address'], as_index=False)['Phone'].transform(fix_anomaly).reset_index(drop=True)

## Task 5: The Final Consolidation

In [6]:
df.drop_duplicates(inplace = True)
display(df.head(10))
display(df.info())

,Name,Address,Zip,Phone
0,Jerry Bartlett,"009 Tristan Meadow Martinezbury, NV",97263.0,(616)477-6740x83474
1,Jennifer Gonzalez,"0098 Finley Valleys Port Michelleshire, ID",97726.0,6094074870
2,Jack Reed,"0098 Finley Valleys Port Michelleshire, ID",97726.0,432.845.9420
3,Mrs. Taylor Johnson DDS,"03284 Carson Fields Acostaview, AK",45268.0,+1-826-533-8606x22246
4,Andrew Martin,"03284 Carson Fields Acostaview, AK",45268.0,705-208-6392
5,Linda Barton,"03284 Carson Fields Acostaview, AK",45268.0,298-984-3837x5183
6,Mrs. Tammy Davis,"048 David Burgs Suite 890 Lake Nancystad, DC",98744.0,+1-562-620-5663
7,Devin Ruiz,"048 David Burgs Suite 890 Lake Nancystad, DC",98744.0,+1-371-732-8951
8,Jacob Todd,"048 David Burgs Suite 890 Lake Nancystad, DC",98744.0,8706775285
9,Kevin Andrews,"0528 Snow Place Michaeltown, NH",82126.0,686-401-2951x1016


<class 'pandas.core.frame.DataFrame'>
Index: 413 entries, 0 to 1523
Data columns (total 4 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   Name     413 non-null    object 
 1   Address  413 non-null    object 
 2   Zip      408 non-null    float64
 3   Phone    412 non-null    object 
dtypes: float64(1), object(3)
memory usage: 16.1+ KB


None

## Task 6: Failure Analysis

In [7]:
rows_with_null = df[df.isnull().any(axis=1)]
display(rows_with_null)

df_save = df.copy()

mask = (df_save["Name"] == "Tammie ThoXXmas")
display(df_save.loc[mask])

mask = (df_save["Name"] == "Charles Smith")
display(df_save.loc[mask])

,Name,Address,Zip,Phone
445,Mary Leonard,"1185 Kathy Overpass Apt. 615 Petersonbury, RI",NaN,560.990.1720
474,Tammie ThoXXmas,"15144 Alexandria Glens South Jason, MN",NaN,922.910.9364
544,Miss Cheryl Mills MD,"796 Helen Summit Port Cynthiaburgh, KS",66410.0,NaN
799,Jason Smith,Unit 7471 Box 5533 DPO AA,NaN,568-675-5729
1074,Maria Li,"7673 Hancock Viaduct Apt. 487 Jonesland, FL",NaN,9448117092
1523,Charles Smith,"140 Larry Center Suite 787 West Charlesbury, MD",NaN,4458820578


,Name,Address,Zip,Phone
474,Tammie ThoXXmas,"15144 Alexandria Glens South Jason, MN",NaN,922.910.9364
562,Tammie ThoXXmas,"45165 Carly Lodge Apt. 451 Kennethberg, MA",57575.0,719.697.0949x27701


,Name,Address,Zip,Phone
12,Charles Smith,"05678 Mccullough Branch Elizabethshire, VA",6636.0,001-882-807-9438
37,Charles Smith,"1144 Martin Villages Apt. 102 Hebertport, NM",22162.0,(302)317-7064x056
93,Charles Smith,"25724 Mary Flat Apt. 920 Jamestown, GA",13131.0,001-486-462-7641
216,Charles Smith,"612 Walter Grove Suite 736 Morrismouth, RI",74843.0,3856773171
323,Charles Smith,"323 Aaron Tunnel West Alex, GA",42138.0,4458820578
377,Charles Smith,USNV Johnson FPO AP,44100.0,+1-547-291-3943x705
1523,Charles Smith,"140 Larry Center Suite 787 West Charlesbury, MD",NaN,4458820578


## Task 7: How to Improve

1. What happened to Tammie ThoXXmas was that the majority were mispelled.  Instead of thefuzz replacing with the correct spelling, it took the one that showed up most even though to a person it would be incorrect.

2. The approach also failed for Charles because there were 8 different Charles Smith in the databases.  The address "908 Kelsey Tunnel Suite 775 Cassandrabury, ID" was written 4 different ways, so none of them ended up as majority, which resulted in the address being completely overwritten.  Then the one Charles Smith with the "new" address had a NaN zip, so majority and filling in from other instances of Charles Smith would not work.

3. In these instances, I would manually go in an look at the data that had a discrepency and compare it to the other information I have.  I would then update it manually as this would be a very specific case of what went wrong.

4. I would not use the majority spelling, but rather I learned about "Levenshtein Distance" which gives a score of probability if they are the same string.  I would not replace a string without the score being above a certain threshold.  The most common one according to what I read seems to be a score of 80.  I had also started before realizing that the homework gave specific algorith for name and address, creating a new column of a string of Name and Address to compare.  I feel like this would help alleviate some of the discrepencies of names being replaced.

## Task 8: Regular Expressions for Data Cleaning

In [8]:
# str.extract

log_entries = pd.Series([
    "[2023-10-26 10:00:00] INFO: User logged in",
    "[2023-10-26 10:05:30] WARNING: Invalid input",
    "[2023-10-26 10:10:15] ERROR: Database connection failed",
    "[2023-10-26 10:12:45] DEBUG: Processing request"
])

regex_pattern = r"^\[(?P<Timestamp>.*?)\]\s(?P<Level>.*?):\s(?P<Message>.*)$"
extracted_logs = log_entries.str.extract(regex_pattern)
display(extracted_logs)

,Timestamp,Level,Message
0,2023-10-26 10:00:00,INFO,User logged in
1,2023-10-26 10:05:30,WARNING,Invalid input
2,2023-10-26 10:10:15,ERROR,Database connection failed
3,2023-10-26 10:12:45,DEBUG,Processing request


In [9]:
# str,replace

text_data = pd.Series([
    "Value is {amount}.",
    "The price is [value].",
    "Cost: (number)",
    "Quantity = <qty>"
])

regex_pattern = r"[\{\[\(<](.*?)[>\)\]\}]"
value_text = "<VALUE>"
standardized_text = text_data.str.replace(regex_pattern, value_text, regex = True)
display(standardized_text)

0        Value is <VALUE>.
1    The price is <VALUE>.
2            Cost: <VALUE>
3       Quantity = <VALUE>
dtype: object

In [10]:
# str.filter

df = pd.DataFrame({
    "order_id": [123, 124, 125, 126],
    "customer_name": ["Alice", "Bob", "Charlie", "Diana"],
    "order_status": ["shipped", "cancelled", "shipped", "delivered"],
    "created_at": ["2021-01-05", "2021-01-06", "2021-01-06", "2021-01-07"],
    "updated_at": ["2021-01-07", "2021-01-07", "2021-01-08", "2021-01-08"]
})

regex_pattern = r"^.*_at"
time_columns = df.filter(regex = regex_pattern, axis = 1)
display(time_columns)

,created_at,updated_at
0,2021-01-05,2021-01-07
1,2021-01-06,2021-01-07
2,2021-01-06,2021-01-08
3,2021-01-07,2021-01-08


In [11]:
# str.contains

order_data = [
    "Order #123 has been shipped on 2021-01-05 (Tuesday)",
    "Order #124 has been cancelled",
    "shipment confirmation #125 on 02/06/2021",
    "Order #126 delivered on 01 07 2021",
    "Canceled order #127, refund pending",
    "order #128 - Shipped 2021/03/10"
]
orders = pd.Series(order_data)

regex_pattern = r"(?:ship|deliver)"

shipped_orders = orders.str.contains(regex_pattern, regex = True, case = False)
display(shipped_orders)

regex_pattern = r"#(?P<Order_Number>\d+).*?(?P<Date>\d{2,4}[-\/\s]\d{2}[-\/\s]\d{2,4})"
order_table = orders.str.extract(regex_pattern).dropna(subset = "Date")
order_table["Date"] = pd.to_datetime(order_table["Date"], errors = "coerce", format = "mixed")
order_table["Shipped"] = shipped_orders
display(order_table)

0     True
1    False
2     True
3     True
4    False
5     True
dtype: bool

,Order_Number,Date,Shipped
0,123,2021-01-05,True
2,125,2021-02-06,True
3,126,2021-01-07,True
5,128,2021-03-10,True
